In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

### Daten

In [2]:
ort = "Magdeburg"
bundesland = "Sachsen-Anhalt"
immobilie = "Grundstuecke" 
art = "kaufen"

In [3]:
driver = webdriver.Firefox()

masterurl = 'https://www.immowelt.de/suche/' + str.lower(ort) + '/' + str.lower(immobilie) + '/' + str.lower(art)

driver.get(masterurl)

time.sleep(3)

soup = BeautifulSoup(driver.page_source, 'html.parser')

active_page = soup.find(class_="Button-c3851 primary-4bda4 navNumberButton-d264f")
active_page = [str(x) for x in active_page]
active_page = int(active_page[0][-8:-7])

pages = soup.find_all(class_="Button-c3851")

last_page = pages[-1]
last_page = [str(x) for x in last_page]
last_page = int(last_page[0][-8:-7])

In [4]:
df_final = pd.DataFrame()
df_final_0 = pd.DataFrame()


while active_page < last_page:

    price = soup.find_all(attrs={"data-test":"price"})
    price = [str(x) for x in price]
    price = [x.replace('<div data-test="price">', '') for x in price]
    price = [x.replace('</div>', '') for x in price]
    price = [x.replace(' €', '') for x in price]
    price = [x.replace('auf Anfrage', '0') for x in price]
    price = [x.replace('.', '')for x in price]
    price = [x.replace(',', '.')for x in price]
    price = [float(x) if x != '' else 0.0 for x in price]

    area = soup.find_all(attrs={"data-test":"area"})
    area = [str(x) for x in area]
    area = [x.replace('<div data-test="area">', '') for x in area]
    area = [x.replace('</div>', '') for x in area]
    area = [x.replace(' m²', '') for x in area]
    area = [float(x) if x != '' else 0.0 for x in area]

    price_per_qm = []

    for index in range(len(price)):
        if price[index] != 0 and area[index] != 0:
            price_per_qm.append(price[index] / area[index])
        else:
            price_per_qm.append(0)

    location = soup.find_all(class_="IconFact-c55e4")
    location = [str(x) for x in location]
    location = [x.replace('<div class="IconFact-c55e4"><i class="sd-icon-6e01d">', '') for x in location]
    location = [x.replace('</span></div>', '') for x in location]
    location = [x.replace('</i><span>', '') for x in location]

    deleteidx = []

    for i in range(len(location)):
        if not "location" in location[i]:
            deleteidx.append(i)

    for index in sorted(deleteidx, reverse=True):
            del location[index]

    location = [x.replace('location', '') for x in location]
    
    obj_id = []
    for a in soup.find_all("a", attrs={"class":"mainSection-88b51 noProject-889ca"}, id=True):
        obj_id.append(a["id"])

    data = {
        "Preis in €" : price,
        "Grundstücksfläche in m²" : area,
        "Preis/m²" : price_per_qm,
        "Ort" : location,
        "ImmoScout24 Objekt Id" : obj_id    
        }

    df = pd.DataFrame(data)

    df_sorted = df[df["Preis/m²"] != 0]
    df_sorted_0 = df[df["Preis/m²"] == 0]

    df_final = pd.concat([df_final, df_sorted])
    df_final_0 = pd.concat([df_final_0, df_sorted_0])

    active_page = soup.find(class_="Button-c3851 primary-4bda4 navNumberButton-d264f")
    active_page = [str(x) for x in active_page]
    active_page = int(active_page[0][-8:-7])

    if active_page < last_page:

        url = masterurl + "?d=true&sd=DESC&sf=TIMESTAMP&sp=" + str(active_page + 1)

        driver.get(url)

        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        url = masterurl

In [5]:
df_final = df_final.reset_index(drop=True)

In [6]:
df_final_filtered = df_final[~df_final["Ort"].str.contains("max.")]
df_final_filtered

,Preis in €,Grundstücksfläche in m²,Preis/m²,Ort,ImmoScout24 Objekt Id
0,149000.0,607.0,245.469522,Berliner Chaussee (Berliner Chaussee),2db625l
1,1100000.0,4496.0,244.661922,Magdeburg (Salbke),2dnbs5k
2,59000.0,540.0,109.259259,Magdeburg (Randau-Calenberge),2drsb5k
3,184140.0,837.0,220.000000,Magdeburg (Beyendorf-Sohlen),2dwxc5j
4,116380.0,529.0,220.000000,Magdeburg (Beyendorf-Sohlen),2drxc5j
5,1180740.0,5367.0,220.000000,Magdeburg (Beyendorf-Sohlen),2dfxc5j
6,160000.0,802.0,199.501247,Magdeburg (Diesdorf),2ddsc5j
7,110000.0,420.0,261.904762,Magdeburg (Stadtfeld West),2dvez5g
8,2448000.0,9792.0,250.000000,Magdeburg (Fermersleben),2dq5b5g
9,700000.0,2663.0,262.861434,Magdeburg (Sudenburg),2djwa5g


In [7]:
df_final_filtered.describe()

,Preis in €,Grundstücksfläche in m²,Preis/m²
count,4.000000e+01,40.000000,40.000000
mean,4.556575e+05,2904.500000,245.913674
std,5.368642e+05,6020.015308,114.665168
min,5.900000e+04,342.000000,19.047619
25%,1.422750e+05,562.500000,202.753010
50%,1.765000e+05,835.500000,250.000000
75%,5.918775e+05,2412.500000,300.000000
max,2.448000e+06,33963.000000,578.947368


In [8]:
df_final_0 = df_final_0.reset_index(drop=True)

In [9]:
df_final_filtered_0 = df_final_0[~df_final_0["Ort"].str.contains("max.")]
df_final_filtered_0

,Preis in €,Grundstücksfläche in m²,Preis/m²,Ort,ImmoScout24 Objekt Id
0,0.0,10460.0,0.0,Magdeburg (Lemsdorf),2dp4z5j
1,0.0,68291.0,0.0,Magdeburg (Berliner Chaussee),2dhwn5h
2,300000.0,0.0,0.0,Magdeburg (Salbke),2dl465g
3,0.0,28765.0,0.0,Magdeburg (Neue Neustadt),2dckd5a
4,0.0,656.0,0.0,Magdeburg / Ottersleben (Ottersleben),2c4xf5n
5,0.0,4612.0,0.0,Magdeburg (Kannenstieg),29n5m4m


In [10]:
driver.quit()